# Implementação 2: Toy Example

Esse notebook apresenta a segunda implementação do Toy Example proposto.
Sua proposta segue os seguintes objetivos:
* Além do modelo implementado antes, ele também conterá métricas e avaliações quanto ao seu desempenho de justiça.
* Assim como o notebook anterior, o seu objetivo é dar prosseguimento ao checklist proposto pelo NIAR.

## Processamento dos dados

As informações quanto ao pré-processamento podem ser encontradas no notebook anterior. Agora, há também um data card descrevendo as propriedades do dataset usado no modelo.

Para essa versão, devido à necessidade de uma análise de justiça do modelo, ficou aparente a necessidade de revisitar algumas características do dataset que foram mal representadas. Para tal, novos atributos foram criados:
* sex_f_share (Razão de presença de mulheres);
* age_15_59_share e age_0_14_share (Razão da presença de diferentes faixas etárias, note que 60 para mais já estava presente no dataset);
* raca_branca_share, raca_negra_share e raca_outras_share (Razão de presença das diferenças raças presentes no dataset);
* O estado (region) onde o hospital está presente (para avaliação de desempenho do modelo por região).

## Documentação

Para essa versão, além do model card e do data card, será feito relatórios sobre a performance do modelo dentro das dimensões da IAR.

* fairness_report.md: Relatório dos passos tomados para verificar e amenizar viéses presentes no modelo.
* ...
* ...

### Etapa 1: Recriação do modelo anterior
A fim de verificar o desempenho do modelo anterior, ele será apenas recriado, porém a avaliação de desempenho dele será feito de forma diferente.

In [1]:
import pandas as pd
import numpy as np
import lightgbm as lgb
import sklearn as sk
from sklearn.metrics import mean_absolute_error, mean_squared_error


df = pd.read_csv("./SIH_Dados/Tabela_lag_Final.csv")

In [3]:
## PROCESSAMENTO DOS DADOS

df.dropna(inplace=True)


X_Train = df[(df.year >= 2022) & (df.year <= 2024)]
X_Train = X_Train[(X_Train.year < 2024) | (X_Train.month <= 6)].reset_index(drop=True)
Y_Train = X_Train["J_count"]
X_Train.drop(columns=["J_count", "region"], inplace=True)

X_Valid = df[(df.year == 2024) & (df.month >= 7)].reset_index(drop=True)
Y_Valid = X_Valid["J_count"]
X_Valid.drop(columns=["J_count", "region"], inplace=True)

X_Test = df[df.year == 2025].reset_index(drop=True)
Y_Test = X_Test["J_count"]

# Salvando para mais tarde
df_resultados = X_Test[["year", "month", "CNES", "hospital_porte", "region"]]

X_Test.drop(columns=["J_count", "region"], inplace=True)

In [4]:
model = sk.linear_model.LinearRegression().fit(X_Train, Y_Train)

val_resul = model.predict(X_Valid)
test_resul = model.predict(X_Test)

In [5]:
seed = 778

lgb_train = lgb.Dataset(X_Train, Y_Train)
lgb_valid = lgb.Dataset(X_Valid, Y_Valid)


model = lgb.train(params={"objective": "poisson"}, train_set=lgb_train, valid_sets=lgb_valid, callbacks=[lgb.early_stopping(stopping_rounds=5)])

res_pred = model.predict(X_Test, num_iteration=model.best_iteration)

[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.014733 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 24420
[LightGBM] [Info] Number of data points in the train set: 56250, number of used features: 105
[LightGBM] [Info] Start training from score 3.504180
Training until validation scores don't improve for 5 rounds
Did not meet early stopping. Best iteration is:
[100]	valid_0's poisson: -91.2195


### Etapa 2: Métricas de Justiça

Para avaliação de Justiça, primeiro agrupa-se as variáveis que representam os grupos presentes no dataset. Note que como os valores são vários, devido aos "lags", decidiu-se agrupá-los pela média.

Os grupos sensíveis que serão analisados são:
* Sexo: Masculino e Feminino;
* Idade: Abaixo dos 15 anos, dos 15 aos 59 e 60 para cima;
* Região: separada por estados;
* Raça: Branca, Preta e outras (Parda, Amarela e Indígena);

Para cada grupos, serão usados as seguintes métricas:
* ...
* ...

In [7]:
# Agrupando as variáveis contendo os grupos sensíveis para avaliação de Justiça
df_resultados["sex_M_share"] = (X_Test["J_sex_m_share_lag1"] + X_Test["J_sex_m_share_lag3"] + X_Test["J_sex_m_share_lag12"]) / 3
df_resultados["sex_F_share"] = (X_Test["J_sex_f_share_lag1"] + X_Test["J_sex_f_share_lag3"] + X_Test["J_sex_f_share_lag12"]) / 3
df_resultados["age_60p_share"] = (X_Test["J_age_60p_share_lag1"] + X_Test["J_age_60p_share_lag3"] + X_Test["J_age_60p_share_lag12"]) / 3
df_resultados["age_15_59_share"] = (X_Test["J_age_15_59_share_lag1"] + X_Test["J_age_15_59_share_lag3"] + X_Test["J_age_15_59_share_lag12"]) / 3
df_resultados["age_0_14_share"] = (X_Test["J_age_0_14_share_lag1"] + X_Test["J_age_0_14_share_lag3"] + X_Test["J_age_0_14_share_lag12"]) / 3
df_resultados["raca_branca_share"] = (X_Test["J_raca_branca_share_lag1"] + X_Test["J_raca_branca_share_lag3"] + X_Test["J_raca_branca_share_lag12"]) / 3
df_resultados["raca_preta_share"] = (X_Test["J_raca_preta_share_lag1"] + X_Test["J_raca_preta_share_lag3"] + X_Test["J_raca_preta_share_lag12"]) / 3
df_resultados["raca_outras_share"] = (X_Test["J_raca_outras_share_lag1"] + X_Test["J_raca_outras_share_lag3"] + X_Test["J_raca_outras_share_lag12"]) / 3

# Salvando os resultados dos modelos e o valor real
df_resultados["Valor Real"] = Y_Test
df_resultados["Base resultados"] = test_resul
df_resultados["LightGBM resultados"] = res_pred

# Cálculo das Métricas